# Quality Model App

## Requirements

### Widget Library

In [ ]:
%pip install ipywidgets==7.7.3

### App Modules

In [ ]:
import os
import random
import hashlib
import uuid
import json
import datetime
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output


## Preparing Data

### Google Drive Mounting
Mount google drive to be used for storing the data.

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

### Specify the Data Location
Specify the URL to fetch from Mega

In [ ]:
# Data URL on Mega
data_url = 'https://mega.nz/file/sBwG1DqZ#NcN7Q97CJPh5DB-tXGb3a4SV6Bubw9xPFRfqPQ4bmw8'
# Destination path for data transfer
transfer_to_path = ''

### Mega Installation

In [ ]:
import urllib.request

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (runSh, loadingAn)

if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

### Transfer Data from Mega to Google Drive

In [ ]:
import os
import subprocess
import contextlib
from functools import wraps
import errno
import signal
import subprocess
import glob

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def latest_file(folder):
  list_of_files = glob.glob(f'{folder}/*') # * means all 
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfer(url):
    cmd = ["mega-get", url, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
if not transfer_to_path:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(10)
def runShT(args):
    return runSh(args, output=True)

transfer(data_url)
tagged_dataset_path = latest_file('./downloads')

## Specify Data Source and Output File

In [ ]:
input_dir = './downloads/Tile_Generator_Genetic_Algo_V1_16x16-2023-23-2--16-01-20'
output_path = 'output.json'

## Functions Definition

### Compute Hash

In [ ]:
import hashlib

def compute_hash(file_path):
    with open(file_path, 'rb') as img_file:
        img_bytes = img_file.read()
    # Compute hash
    hasher = hashlib.sha256()
    hasher.update(img_bytes)
    hash_id = hasher.hexdigest()
    hasher = hashlib.sha256()
    # hasher.update(img.tobytes())
    # hash_id = hasher.hexdigest()
    return hash_id


## Run on Data Source

### Creating Dictionary from Input Data 

In [ ]:
''' Data dictionay creator function'''
def create_input_data_dict(input_dir):

    # Placeholder for dict to contain result from running on data source.
    data_dict = {}

    print ('[INFO] Running on Data Source...')

    # Walking thru files
    for root, _, files in os.walk(input_dir):
        for file in tqdm(files):
            # Get file path
            file_path = f'{root}/{file}'
            # Check if file is png or jpg
            if os.path.splitext(file_path)[-1] == '.png' or os.path.splitext(file_path)[-1] == '.jpg':
                try:
                    # Compute hash
                    hash_id = compute_hash(file_path)
                    data_dict[hash_id]={'file_path':file_path, 'file_name':file}
                except Exception as e:
                    print [f'[WARNING] Error when processing file: {e}']
            
    # Number of images
    n_images = len(data_dict)
    print (f'[INFO] Completed. Number of images: {n_images}')

    return data_dict

data_dict = create_input_data_dict(input_dir)

## Widgets (Start the App)

In [ ]:
qualityModelWidgets = QualityModelWidgets(data_dict, output_path)
qualityModelWidgets.start()